In [1]:
#from TwitterAPI import TwitterAPI #not the best one
from twython import Twython # This one is supposed to be the best
from twython import TwythonStreamer

import argparse
import twitter
import json
import os.path
import signal
import sys

In [9]:
class autorizador:
    def __init__(self, CK, CKS, AT, ATS):
        self.ConsumerKey = CK
        self.ConsumerKeySecret = CKS
        self.AccessToken = AT
        self.AccessTokenSecret = ATS

    def getConsumerKey(self):
        return self.ConsumerKey

    def getConsumerKeySecret(self):
        return self.ConsumerKeySecret

    def getAccessToken(self):
        return self.AccessToken

    def getAccessTokenSecret(self):
        return self.AccessTokenSecret

def makeAuth(infoStr):
    # infoStr is a tab-separated line: name hours qpoints
    # returns a corresponding Student object
    CK, CKS, AT, ATS = infoStr.split(",")
    return autorizador(CK, CKS, AT, ATS)

def get_creds(filename):
    # open the input file for reading
    infile = open(filename, 'r')
    c = makeAuth(infile.readline())
    for line in infile:
        s = makeAuth(line)
    infile.close()
    return c

# OAuth magic
creds = get_creds('secrets.txt')
oauth_token, oauth_secret, consumer_key, consumer_secret =  creds.AccessToken,creds.AccessTokenSecret,creds.ConsumerKey, creds.ConsumerKeySecret
auth=twitter.OAuth(oauth_token, oauth_secret, consumer_key, consumer_secret)

In [13]:
def save_tweet(tweet, f):
    # Replace HTML entities
    tweet["text"] = tweet["text"].replace("&gt;", ">").replace("&lt;", "<").replace("&amp;", "&")
    json.dump(tweet, f)

t = twitter.Twitter(auth=auth)
users = ['aurelionuno']
outfile = open('data.json', 'w')

In [14]:
def save_user_tweets(user, n, auth):
    t = twitter.Twitter(auth=auth)
    print("Fetching %i tweets from @%s" % (n, user))
    tweets = t.statuses.user_timeline(screen_name=user, count=n)
    print("  (actually fetched %i)" % len(tweets))
    for tweet in tweets:
        save_tweet(tweet, outfile)
        print(tweet['text'])

In [15]:
save_user_tweets('aurelionuno', 10, auth)

Fetching 10 tweets from @aurelionuno
  (actually fetched 10)
Porque la educación empieza en casa, mi reconocimiento a todas las mamás de México. #DíaDeLasMadres https://t.co/4KaAfbXLOV
¡Muchas felicidades a todas las mamás en su día! #DíaDeLasMadres https://t.co/Y9iupyVdhD
Agradezco al Director Antonio y a la comunidad educativa por permitirme dialogar con ustedes. #VisitaEnLunes https://t.co/Ly2jd1AJ47
En vacaciones arrancaremos las #EscuelasDeVerano para que los alumnos aprovechen el tiempo divirtiéndose y aprendiendo.
Abriremos más #EscuelasdeTiempoCompleto. Hoy tenemos más de 25,000 en todo el país. https://t.co/uj2IRfxKwL
A partir de ahora, las estancias infantiles de SEDESOL, además de cuidar a sus hijos, ofrecerán educación preescolar https://t.co/3gK4WEYHk3
Desde la @SEP_mx, promovemos acciones para apoyar a las mamás, como aumentar el horario de todos los preescolares de 3 a 4 horas.
3 de cada 4 miembros de los Consejos Escolares de Participación Social de las escuelas del paí

### Twitter documentation: POST statuses/filter

Returns public statuses that match one or more filter predicates. Multiple parameters may be specified which allows most clients to use a single connection to the Streaming API. Both GET and POST requests are supported, but GET requests with too many parameters may cause the request to be rejected for excessive URL length. Use a POST request to avoid long URLs.

The track, follow, and locations fields should be considered to be combined with an OR operator. track=foo&follow=1234 returns Tweets matching “foo” OR created by user 1234.

The default access level allows up to 400 track keywords, 5,000 follow userids and 25 0.1-360 degree location boxes. If you need elevated access to the Streaming API, you can contact Gnip."""


In [7]:
def save_tweet(tweet, f):
    # Replace HTML entities
    tweet["text"] = tweet["text"].replace("&gt;", ">").replace("&lt;", "<").replace("&amp;", "&")
    json.dump(tweet, f)

t = twitter.Twitter(auth=auth)
users = ['aurelionuno']
outfile = open('data.json', 'w')

def save_user_tweets(user, n, auth):
    t = twitter.Twitter(auth=auth)
    print("Fetching %i tweets from @%s" % (n, user))
    tweets = t.statuses.user_timeline(screen_name=user, count=n)
    print("  (actually fetched %i)" % len(tweets))
    for tweet in tweets:
        save_tweet(tweet, outfile)
        print(tweet['text'])

def get_tweets_from_stream(filter_, n, auth):
  
    # Connect to the stream
        twitter_stream = twitter.TwitterStream(auth=auth)
    
        if filter_ is None and filters_file is None:
            stream = twitter_stream.statuses.sample()
        else:
            if filter_ is not None:
                track = filter_
            elif filters_file is not None:
                track = ",".join(filters_file.read().strip().split("\n"))

            stream = twitter_stream.statuses.filter(track=track)
    
        # Fetch the tweets
        fetched = 0
    
        if num_tweets > 0:
            if outf != sys.stdout: print("Fetching %i tweets... " % num_tweets)
        else:
            signal.signal(signal.SIGINT, signal_handler)
            now = datetime.now().isoformat(sep=" ")
            msg = "[{}] Fetching tweets. Press Ctrl+C to stop.".format(now)
            if outf != sys.stdout: print(msg)
    
        for tweet in stream:
            # The public stream includes tweets, but also other messages, such
            # as deletion notices. We are only interested in the tweets.
            # See: https://dev.twitter.com/streaming/overview/messages-types
            if tweet.has_key("text"):
                # We also only want English tweets
                if tweet["lang"] == "en":
                    save_tweet(tweet, outf, format)
                    fetched += 1
                    if fetched % 100 == 0:
                        now = datetime.now().isoformat(sep=" ")
                        msg = "[{}] Fetched {:,} tweets.".format(now, fetched)
                        if outf != sys.stdout: print(msg)
                    if num_tweets > 0 and fetched >= num_tweets:
                        break
    


outfile.close()